In [1]:
'''This is a test file for this reporitory.
   We would import functions and test in isolation.
'''
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import numpy as np
import unittest
import torch
import torch.nn.functional as F
from model.box.box_wrapper import BoxTensor
from model.box.modules import BoxEmbedding
from model.box_model import BoxRec
from model.vector_model import MatrixFactorizationWithBias, MatrixFactorization
from trainer import Trainer
from eval_metrics import AP, NDCG, MRR, MAP, precision_At_k, recall_At_k
from matplotlib import pyplot as plt

In [35]:
class Boxtest(unittest.TestCase):
    def test_addition(self):
        self.assertEqual(1 + 1, 2)
    def test_box_tensor(self):
        box = BoxTensor(torch.FloatTensor([[1, 2, 4],[3, 4, 5]]))
        self.assertTrue(torch.equal(box.z, torch.tensor([1, 2, 4])))
        self.assertTrue(torch.equal(box.Z, torch.tensor([3, 4, 5])))
        tensor1 = torch.FloatTensor([[1, 2, 4],[3, 4, 5]])
        tensor2 = torch.FloatTensor([[1, 2, 4],[3, 4, 5]])
        tensor3 = torch.FloatTensor([[1, 2, 4],[3, 4, 5]])
        tensor_stack = torch.stack([tensor1, tensor2, tensor3], dim=0)
        box = BoxTensor(tensor_stack)
        self.assertTrue(torch.equal(box.z, tensor_stack[:,0]))
        self.assertTrue(torch.equal(box.Z, tensor_stack[:,1]))
    
    def test_intersection(self):
        box1 = BoxTensor(torch.FloatTensor([[1, 2, 4],[3, 4, 5]]))
        box2 = BoxTensor(torch.FloatTensor([[2, 3, 4],[3, 4, 5]]))
        self.assertTrue(torch.equal(box1.intersection(box2).z, torch.FloatTensor([2, 3, 4])))
        self.assertTrue(torch.equal(box1.intersection(box2).Z, torch.FloatTensor([3, 4, 5])))
    
    def test_gumbel_intersection(self):
        box1 = BoxTensor(torch.FloatTensor([[1, 2, 4],[3, 4, 5]]))
        box2 = BoxTensor(torch.FloatTensor([[2, 3, 4],[3, 4, 5]]))
        intersection_z, intersection_Z = box1._intersection(box2, intersection_temp=0.00001, bayesian=True)
        self.assertTrue(torch.allclose(intersection_z, torch.FloatTensor([2, 3, 4])))
        self.assertTrue(torch.allclose(intersection_Z, torch.FloatTensor([3, 4, 5])))
    
    def test_gumbel_intersection_log_volume(self):
        volume_temp = 10
        intersection_temp = 0.0000001
        box1 = BoxTensor(torch.FloatTensor([[1, 2, 4],[3, 4, 5]]))
        box2 = BoxTensor(torch.FloatTensor([[2, 3, 4],[3, 4, 5]]))
        # self volume test through intersection.
        volume_box_1 = box1._log_soft_volume_adjusted(box1.z,
                                                box1.Z,
                                                volume_temp=volume_temp,
                                                intersection_temp=intersection_temp)
        diff = box1.Z - box1.z - 2 * 0.57721566490153286060 * intersection_temp
        calculated_volume = torch.log(F.softplus(torch.FloatTensor(diff), beta=volume_temp)).sum()
        self.assertTrue(torch.allclose(volume_box_1, torch.FloatTensor(calculated_volume)))

        volume_box_2 = box2._log_soft_volume_adjusted(box2.z,
                                                box2.Z,
                                                volume_temp=volume_temp,
                                                intersection_temp=intersection_temp)
        diff = box2.Z - box2.z - 2 * 0.57721566490153286060 * intersection_temp
        calculated_volume = torch.log(F.softplus(torch.FloatTensor(diff), beta=volume_temp)).sum()
        # self.assertTrue(torch.allclose(volume_box_2, torch.FloatTensor(calculated_volume)))

        intersection_log_volume = box1.gumbel_intersection_log_volume(box2,
                                                                    volume_temp=volume_temp,
                                                                    intersection_temp=intersection_temp)
        
        self.assertTrue(torch.allclose(intersection_log_volume, volume_box_2))

        box1 = BoxTensor(torch.FloatTensor([[1, 3, 4],[3, 4, 5]]))
        box2 = BoxTensor(torch.FloatTensor([[2, 2, 4],[9, 6, 7]]))
        intersection_log_volume_2 = box1.gumbel_intersection_log_volume(box2,
                                                                    volume_temp=volume_temp,
                                                                    intersection_temp=intersection_temp)
        self.assertTrue(torch.allclose(intersection_log_volume, intersection_log_volume_2))

    def test_box_embedding_layer(self):
        #init test
        box_embedding = BoxEmbedding(10, 20)
        # all_function test
        self.assertTrue(box_embedding.all_boxes.z.shape == (10, 20))
        self.assertTrue(box_embedding.all_boxes.Z.shape == (10, 20))
        self.assertTrue(box_embedding.all_boxes.z.requires_grad)
        self.assertTrue(box_embedding.all_boxes.Z.requires_grad)
        # forward test
        self.assertTrue(box_embedding(torch.LongTensor([0, 6])).z.shape == (2,20))
        self.assertTrue(box_embedding(torch.LongTensor([1, 2])).z.requires_grad)

        #get_volume test
        self.assertIsNotNone(box_embedding.get_volumes(volume_temp=10))

    def test_box_rec(self):
        #init test
        box_rec = BoxRec(10, 20, box_type="BoxTensor", volume_temp=10, intersection_temp=0.00, embedding_dim=20)
        self.assertTrue(box_rec.n_users == 10)
        self.assertTrue(box_rec.n_items == 20)
        self.assertTrue(box_rec.volume_temp == 10)
        self.assertTrue(box_rec.intersection_temp == 0.00)
        self.assertTrue(box_rec.embedding_dim == 20)
        self.assertTrue(box_rec.box_type == "BoxTensor")
        #forward test soft box
        similarity = box_rec.forward(torch.LongTensor([1]), torch.LongTensor([2]))
        user = box_rec.user_embeddings(torch.LongTensor([1]))
        item = box_rec.item_embeddings(torch.LongTensor([2]))
        min_z = torch.max(user.z, item.z)
        max_Z = torch.min(user.Z, item.Z)
        vol = torch.log(F.softplus(max_Z - min_z, beta=10)).sum()
        self.assertTrue(torch.allclose(similarity, vol))

        #forward test gumbel box
        intersection_temp = 0.1
        box_rec = BoxRec(10, 20, box_type="BoxTensor", volume_temp=10, intersection_temp=intersection_temp, embedding_dim=20)
        similarity = box_rec.forward(torch.LongTensor([1]), torch.LongTensor([2]))
        user = box_rec.user_embeddings(torch.LongTensor([1]))
        item = box_rec.item_embeddings(torch.LongTensor([2]))
        min_z = intersection_temp * torch.log(torch.exp(user.z / intersection_temp) + torch.exp(item.z / intersection_temp))
        max_Z = -intersection_temp * torch.log(torch.exp(-user.Z / intersection_temp) + torch.exp(-item.Z / intersection_temp))
        vol_gumbel = torch.log(F.softplus(max_Z - min_z - 2 * 0.57721566490153286060 * intersection_temp , beta=10)).sum()
        self.assertTrue(torch.allclose(similarity, vol_gumbel))

        intersection_temp = 0.0001
        box_rec = BoxRec(10, 20, box_type="BoxTensor", volume_temp=10, intersection_temp=intersection_temp, embedding_dim=20)
        similarity = box_rec.forward(torch.LongTensor([1]), torch.LongTensor([2]))
        user = box_rec.user_embeddings(torch.LongTensor([1]))
        item = box_rec.item_embeddings(torch.LongTensor([2]))

        min_z = intersection_temp * torch.logaddexp(user.z / intersection_temp, item.z / intersection_temp)
        max_Z = -intersection_temp * torch.logaddexp(-user.Z / intersection_temp, -item.Z / intersection_temp)
        vol_gumbel = torch.log(F.softplus(max_Z - min_z - 2 * 0.57721566490153286060 * intersection_temp , beta=10)).sum()
        vol_gumbel, similarity
        self.assertTrue(torch.allclose(similarity, vol_gumbel))


# Run Tests

In [36]:
# Run the tests on a jupyter notebook
unittest.main(argv=[''], exit=False)

.......
----------------------------------------------------------------------
Ran 7 tests in 0.119s

OK
